In [ ]:
# A simple DTW Notebook

In [1]:
# do all the imports
%matplotlib inline

import sys, os
import numpy as np
import pandas as pd
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

In [2]:
# graphical and print preferences
ldesign = 50
cmap = sns.light_palette("caramel",ldesign,input="xkcd")
cmap20 = cmap[0:20] 
pd.reset_option('display.float_format')
pd.set_option('precision',3)

In [88]:
observation = "d o e s"
reference = "d o s e s"
ins_cost = 1.5
del_cost = 1.
sub_cost = 1.5

In [149]:
def dtw(obs="",ref="",iC=1.,dC=1.,sC=1.):
    x = np.append('/',obs.split())
    y = np.append('/',ref.split())
    # add dummy symbol in front to accomodate initial insertions and deletions
#    print(x,y) 
    trellis = np.zeros((len(y),len(x)))
    bptr = np.zeros((len(y),len(x),2),dtype='int')

    # initialize full row of deletions and insertions
    for ix in range(1,len(x)): 
        trellis[0,ix] = trellis[0,ix-1] + iC
        bptr[0,ix] = [0,ix-1]
    for iy in range(1,len(y)): 
        trellis[iy,0] = trellis[iy-1,0] + dC
        bptr[iy,0] = [iy-1,0]
            
    for ix in range(1,len(x)):
        for iy in range(1,len(y)):
 
            trellis[iy,ix] = trellis[iy-1,ix-1] + sC * int(x[ix] != y[iy])
            bptr[iy,ix] = [iy-1,ix-1]
        
            dScore = trellis[iy-1,ix] + dC
            if( dScore < trellis[iy,ix] ):
                trellis[iy,ix] = dScore
                bptr[iy,ix] = [iy-1,ix]
        
            iScore = trellis[iy,ix-1] + iC 
            if( iScore < trellis[iy,ix] ):
                trellis[iy,ix] = iScore
                bptr[iy,ix] = [iy,ix-1]
                
    print(pd.DataFrame(trellis))
#    print(bptr)
    ixback = len(x)-1
    iyback = len(y)-1
    btrace = [y[iyback],x[ixback]]
    while((ixback > 0) & (iyback >0)):
#        print("%s y%d   --  %s x%d " % (y[iyback],iyback,x[ixback],ixback) )
        (iyback,ixback) = bptr[iyback,ixback]
        btrace = np.vstack([[y[iyback],x[ixback]],btrace])
        
    print(pd.DataFrame(btrace[1:6,:].T,index=['ref','obs']))

In [150]:
observation = "d o e s"
reference = "d o s i s"
dtw(obs=observation, ref=reference, iC=1.5,dC=1.5)

     0    1    2    3    4
0  0.0  1.5  3.0  4.5  6.0
1  1.5  0.0  1.5  3.0  4.5
2  3.0  1.5  0.0  1.5  3.0
3  4.5  3.0  1.5  1.0  1.5
4  6.0  4.5  3.0  2.5  2.0
5  7.5  6.0  4.5  4.0  2.5
     0  1  2  3  4
ref  d  o  s  i  s
obs  d  o  o  e  s


In [103]:
ix=0
iy = 3
bb = np.zeros((3,3,2))
bb[1,2] = [3,7]
print(bb[1,2])

[3. 7.]


In [104]:
(ix,iy)=bb[1,2]

In [120]:
btr = np.empty((1,2))
btr = np.vstack([btr,['o','q']])
print(btr)

[['2.46151511809e-312' '2.39785524434e-312']
 ['o' 'q']]


In [116]:
x=np.vstack([[1,2,3],[4,5,6]])
print(x)

[[1 2 3]
 [4 5 6]]
